In [1]:
from bs4 import BeautifulSoup
import requests
import re 

In [2]:
# This is the piece of code to scrape all of the urls - takes about 7mins without scraping

# full_list_url = []

# for i in range(1,333):
#     base_url = f"https://www.immoweb.be/en/search/house/for-sale?countries=BE&page={i}&orderBy=relevance"
#     r = requests.get(base_url)
#     soup = BeautifulSoup(r.content, "html.parser")
    
#     for elem in soup.find_all("a", attrs={"class":"card__title-link"}):
#          full_list_url.append(elem.get('href'))
#          with open("full_list.txt", "a") as f:
#             f.write(elem.get('href'))
#             f.write('\n')
            
# print("Written to text file Successfully!")
# print(len(full_list_url))

In [19]:
# TEST THREADING
import requests
from bs4 import BeautifulSoup
import threading
import time

# Function to scrape URLs
def scrape_urls(page_num):
    base_url = f"https://www.immoweb.be/en/search/house/for-sale?countries=BE&page={page_num}&orderBy=relevance"
    r = requests.get(base_url)
    soup = BeautifulSoup(r.content, "html.parser")
    
    urls = []
    for elem in soup.find_all("a", attrs={"class": "card__title-link"}):
        urls.append(elem.get('href'))
    
    button = soup.find_all("span", attrs={"class": "button__label"})
        
    # Save URLs to file - full_list.txt (local storage)
    with open("full_list.txt", "a") as f:
        for url in urls:
            f.write(url + '\n')
    return button

def thread_scraping():
    full_list_url = []
    num_pages = 333

    # Create a list to store threads
    threads = []
    start_time = time.time()  # Start timer
    
    # Create and start threads
    for i in range(1, num_pages + 1):
        t = threading.Thread(target=lambda: full_list_url.extend(scrape_urls(i)))
        threads.append(t)
        t.start()

    # Wait for all threads to complete and then join
    for t in threads:
        t.join()

    end_time = time.time()  # Stop timer
    execution_time = end_time - start_time

    print("Scraping completed!")
    print("Total URLs scraped:", len(full_list_url))
    print("Total time:", execution_time, "seconds")

button = scrape_urls(1)



In [20]:
for elem in button:
    print(elem)

<span class="button__label">Log in</span>


In [4]:
import requests
from bs4 import BeautifulSoup as soup
import re
import json

#https://www.immoweb.be/en/classified/house/for-sale/libin/6890/10657263        

url = "https://www.immoweb.be/en/classified/apartment/for-sale/deinze/9800/10663064"
house_page = requests.get(url)
house_page = soup(house_page.text, 'html.parser')

regex = r"window.classified = (\{.*\})"

script = house_page.find('div',attrs={"id":"main-container"}).script.text
script = re.findall(regex, script)
 
script = json.loads(script[0])
#print(script['property']['location']['locality'])

In [5]:
def scrape_house(url):
    """Scrapes all the info from a house listing"""

    # Get the house listing and make a soup
    house_page = requests.get(url)
    house_page = soup(house_page.text, 'html.parser')
    final_dictionary = {}

    # Get the hidden info from the java script
    regex = r"window.classified = (\{.*\})"
    script = house_page.find('div',attrs={"id":"main-container"}).script.text
    script = re.findall(regex, script)

    final_dictionary = {}
        #Locality
    try:
        script = json.loads(script[0])
    except:
        return {}
        #Locality
    
    try:
        final_dictionary['locality'] = script['property']['location']['locality']
    except:
        final_dictionary['locality'] = 'UNKNOWN'
    #type of property
    try:
        final_dictionary['type of property'] = script['property']['type']
    except:
        final_dictionary['type of property'] = 'UNKNOWN'
    #subtype of property
    try:
        final_dictionary['subtype of property'] = script['property']['subtype']
    except:
        final_dictionary['subtype of property'] = 'UNKNOWN'
    #price
    try:
        final_dictionary['price'] = script['price']['mainValue']
    except:
        final_dictionary['price'] = 'UNKNOWN'
    #- Number of rooms
    try:
        final_dictionary['number_rooms'] = script['property']['bedroomCount']
    except:
        final_dictionary['number_rooms'] = 'UNKNOWN'
    
    # living area
    try:
        final_dictionary['living_area'] = script['property']['netHabitableSurface']
    except:
        final_dictionary['living_area'] = 'UNKNOWN'
    # Fully equipped kitchen (Yes/No)
    try:
        final_dictionary['kitchen'] = script['property']['kitchen']['type']
    except:
        final_dictionary['kitchen'] = 0

    # NOT INSTALLED / INSTALLED 
    # Furnished (Yes/No)
    try:
        final_dictionary['furnished'] = script['transaction']['sale']['isFurnished']
    except:
        final_dictionary['furnished'] = 'UNKNOWN'
    # Open fire (Yes/No)
    try:
        final_dictionary['fireplace'] = script['property']['fireplaceCount']
    except:
        final_dictionary['fireplace'] = 0

    # Terrace (Yes/No)
    try:
        final_dictionary['terrace'] = script['property']['hasTerrace']
    except:
        final_dictionary['terrace'] = 0
    # If yes: Area
    try:
        final_dictionary['terrace_area'] = script['property']['terraceSurface']
    except: 
        final_dictionary['terrace_area'] = 0
    # Garden
    try:
        final_dictionary['garden'] = script['property']['hasGarden']
    except:
        final_dictionary['garden'] = 0
    #- If yes: Area
    try:
        final_dictionary['garden_area'] = script['property']['gardenSurface']
    except:
        final_dictionary['garden_area'] = 0
    # Surface of the land
    try: 
        final_dictionary['surface_land'] = script['property']['land']['surface']
    except:
        final_dictionary['surface_land'] = "UNKNOWN"
    # Surface area of the plot of land - TO ASK
    # Number of facades
    try:
        final_dictionary['number_facades'] = script['property']['building']['facadeCount']
    except:
        final_dictionary['number_facades'] = "UNKNOWN"
    # Swimming pool (Yes/No)
    try:
        final_dictionary['swimming_pool'] =  script['property']['hasSwimmingPool']
    except:
        final_dictionary['swimming_pool'] = 0
    # State of the building (New, to be renovated, ...)
    try:
        final_dictionary['building_state'] = script['property']['building']['condition']
    except:
        final_dictionary['building_state'] = 'UNKNOWN'

    return final_dictionary

In [6]:
# import requests
# from bs4 import BeautifulSoup as soup
# import re
# import json
# from concurrent.futures import ThreadPoolExecutor,ProcessPoolExecutor, as_completed
# import time
# import pandas as pd

# houses_links = []

# with ThreadPoolExecutor(max_workers=10) as executor:
#     futures = [executor.submit(scrape_house, url) for url in houses_links]
#     results =  [item.result() for item in futures]
#     df = pd.DataFrame(results)

In [7]:
import pandas as pd
from concurrent.futures import ThreadPoolExecutor,ProcessPoolExecutor, as_completed

def create_dataframe():
    houses_links = []
    with open("./full_list_20k.txt", "r") as f:
        count = 0
        for url in f:
            if count < 300:
                houses_links.append(url)
                count +=1
            else:
                break

    with ThreadPoolExecutor(max_workers=10) as executor:
        try:
            futures = [executor.submit(scrape_house, url) for url in houses_links]
            results =  [item.result() for item in futures]
            df = pd.DataFrame(results)
        except:
            print("BREAK! Writing scraped records to csv")
            df.to_csv('dataframe.csv', index = True)
            return df
    return df


In [8]:
df = create_dataframe()

In [9]:
df.to_csv('dataframe.csv', index=True)

In [10]:
df = df.drop_duplicates()
df

,locality,type of property,subtype of property,price,number_rooms,living_area,kitchen,furnished,fireplace,terrace,terrace_area,garden,garden_area,surface_land,number_facades,swimming_pool,building_state
0,Libin,HOUSE,HOUSE,299900.0,3.0,124.0,NOT_INSTALLED,False,NaN,True,8.0,True,257.0,320,3,False,AS_NEW
1,Rixensart,HOUSE,HOUSE,399000.0,3.0,173.0,SEMI_EQUIPPED,False,NaN,True,15.0,True,333.0,428,3,False,TO_RENOVATE
2,Rixensart Genval,HOUSE,HOUSE,485000.0,3.0,148.0,SEMI_EQUIPPED,False,NaN,None,NaN,True,88.0,232,3,False,GOOD
3,Waterloo,HOUSE,HOUSE,549000.0,4.0,207.0,INSTALLED,False,NaN,True,25.0,True,230.0,464,3,False,GOOD
4,Braine-l'Alleud,HOUSE,HOUSE,549000.0,4.0,207.0,INSTALLED,False,NaN,True,25.0,True,230.0,464,3,False,GOOD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,Sint-Laureins,HOUSE,HOUSE,299000.0,3.0,127.0,0,False,NaN,None,NaN,None,NaN,1593,3,False,GOOD
296,Zelzate,APARTMENT,APARTMENT,189000.0,2.0,90.0,0,False,NaN,None,NaN,None,NaN,UNKNOWN,2,False,None
297,Zelzate,APARTMENT,APARTMENT,169000.0,2.0,90.0,None,False,NaN,None,NaN,True,6.0,None,2,False,TO_BE_DONE_UP
298,Oostende,APARTMENT,APARTMENT,285000.0,3.0,81.0,INSTALLED,False,NaN,None,NaN,None,NaN,None,2,False,AS_NEW


In [11]:
rows_to_remove = []
for index, row in df.iterrows():
    if row.isnull().all():
        rows_to_remove.append(index)

# Remove the marked rows from the DataFrame
df = df.drop(rows_to_remove)
df.to_csv('cleaned.csv', index=True)

In [12]:
df['fireplace'] = df['fireplace'].fillna(0)
df['fireplace'] = df['fireplace'].replace(-1.0, 0)
df['fireplace'].unique()

array([0., 2., 1.])

In [13]:
print(df)

             locality type of property subtype of property     price  \
0               Libin            HOUSE               HOUSE  299900.0   
1           Rixensart            HOUSE               HOUSE  399000.0   
2    Rixensart Genval            HOUSE               HOUSE  485000.0   
3            Waterloo            HOUSE               HOUSE  549000.0   
4     Braine-l'Alleud            HOUSE               HOUSE  549000.0   
..                ...              ...                 ...       ...   
295     Sint-Laureins            HOUSE               HOUSE  299000.0   
296           Zelzate        APARTMENT           APARTMENT  189000.0   
297           Zelzate        APARTMENT           APARTMENT  169000.0   
298          Oostende        APARTMENT           APARTMENT  285000.0   
299            Bilzen        APARTMENT           PENTHOUSE  265000.0   

     number_rooms  living_area        kitchen furnished  fireplace terrace  \
0             3.0        124.0  NOT_INSTALLED     False  